In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import csv

In [4]:
df = pd.read_csv('main_task.csv')
df.loc[df['City'] == 'Oporto', 'City'] = 'Porto'

In [5]:
cities = df.City.unique()

In [6]:
from tqdm import tqdm_notebook as tqdmn

In [62]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(options=options)

cities_pars = []

for city in cities:
    browser.get('https://yandex.com/maps/')
    assert 'Yandex' in browser.title

    elem = browser.find_element_by_class_name(
            'input__control')  # Find the search box
    try:
        elem.send_keys(city + Keys.RETURN)
        time.sleep(0.5)
        cities_pars.append({'City': city,
                            'country': browser.find_element_by_xpath('/html/body/div[1]/div[2]/div[4]/div[1]/div[1]/div[1]/div/div[1]/div/div[5]/div[3]/div/div[3]/div/a[2]').get_attribute('title'),
                            'city_coord': browser.find_element_by_class_name(
                                        'toponym-card-title-view__coords-badge').text.split(sep=', ')})
    except:
        browser.get('https://yandex.com/maps/')
        elem = browser.find_element_by_class_name('input__control')
        elem.send_keys('City ' + city + Keys.RETURN)
        time.sleep(0.5)
        cities_pars.append({'City': city,
                            'country': browser.find_element_by_xpath('/html/body/div[1]/div[2]/div[4]/div[1]/div[1]/div[1]/div/div[1]/div/div[5]/div[3]/div/div[3]/div/a[2]').get_attribute('title'),
                            'city_coord': browser.find_element_by_class_name(
                                        'toponym-card-title-view__coords-badge').text.split(sep=', ')})
browser.quit()

In [58]:
def pars():    
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    browser = webdriver.Chrome(chrome_options=options)

    cities_pars = []

    for city in cities:
        browser.get('https://yandex.com/maps/')
        assert 'Yandex' in browser.title

        elem = browser.find_element_by_class_name(
                'input__control')  # Find the search box
        try:
            elem.send_keys(city + Keys.RETURN)
            time.sleep(0.5)
            cities_pars.append({'City': city,
                                'country': browser.find_element_by_xpath('/html/body/div[1]/div[2]/div[4]/div[1]/div[1]/div[1]/div/div[1]/div/div[5]/div[3]/div/div[3]/div/a[2]').get_attribute('title'),
                                'city_coord': browser.find_element_by_class_name(
                                            'toponym-card-title-view__coords-badge').text.split(sep=', ')})
        except:
            browser.get('https://yandex.com/maps/')
            elem = browser.find_element_by_class_name('input__control')
            elem.send_keys('City ' + city + Keys.RETURN)
            time.sleep(0.5)
            cities_pars.append({'City': city,
                                'country': browser.find_element_by_xpath('/html/body/div[1]/div[2]/div[4]/div[1]/div[1]/div[1]/div/div[1]/div/div[5]/div[3]/div/div[3]/div/a[2]').get_attribute('title'),
                                'city_coord': browser.find_element_by_class_name(
                                            'toponym-card-title-view__coords-badge').text.split(sep=', ')})
    browser.quit()

In [43]:
pd.DataFrame(cities_pars)

,City,country,city_coord
0,Paris,France,"[48.856663, 2.351556]"
1,Stockholm,Sweden,"[59.325000, 18.070897]"
2,London,United Kingdom,"[51.507351, -0.127660]"
3,Berlin,Germany,"[52.518654, 13.374711]"
4,Munich,Germany,"[48.137187, 11.575691]"
5,Porto,Portugal,"[41.157963, -8.629154]"
6,Milan,Italy,"[45.469436, 9.180621]"
7,Bratislava,Slovakia,"[48.143488, 17.108361]"
8,Vienna,Austria,"[48.206481, 16.363451]"
9,Rome,Italy,"[41.902689, 12.496176]"


In [64]:
import requests
from bs4 import BeautifulSoup
import re

URL = 'https://www.tripadvisor.com' + df['URL_TA']
HEADERS = {'user-agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.152 YaBrowser/21.2.2.101 Yowser/2.5 Safari/537.36',
           'accept': '*/*'}
rest_pars = []


def get_html(url, params=None):
    r = requests.get(url, headers=HEADERS, params=params)
    return r


def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    pattern = re.compile('/d+[.]/d+[,]/d+[.]/d+')
    meals_find = soup.find('div', string='Meals')
    coord_find = soup.find(href="#MAPVIEW")
#     print(coord_find)
    if meals_find:
        meals_find = meals_find.find_next('div').get_text().split(', ')
    else:
        meals_find = 'no_info'

    if coord_find:
        coord_find = coord_find.get_text()
    else:
        coord_find = 'no_info'

    rest_pars.append({'meals': meals_find,
                      'coord_rest': coord_find
                      })


def parse(url):
    html = get_html(url)
    if html.status_code == 200:
        get_content(html.text)
    else:
        print('Error')


for i in URL[:5]:
    parse(i)
    
rest_pars

[{'meals': ['Lunch', 'Dinner', 'Brunch', 'Breakfast', 'Late Night'],
  'coord_rest': '17 boulevard Saint Jacques Paris Marriott Rive Gauche Hotel & Conference Center, 75014 Paris France'},
 {'meals': 'no_info',
  'coord_rest': '39 Aarstavaegen, Stockholm 120 52 Sweden'},
 {'meals': ['Lunch', 'Dinner', 'Late Night', 'Drinks'],
  'coord_rest': '30 North Audley Street Mayfair, London W1K 6ZF England'},
 {'meals': 'no_info', 'coord_rest': 'Berliner Str. 54, 13189 Berlin Germany'},
 {'meals': ['Dinner', 'Lunch'],
  'coord_rest': 'Zielstattstr. 6, 81379 Munich, Bavaria Germany'}]

In [68]:
import requests
from bs4 import BeautifulSoup
import re
import time
URL = 'https://www.tripadvisor.com' + df['URL_TA']
HEADERS = {'user-agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.152 YaBrowser/21.2.2.101 Yowser/2.5 Safari/537.36',
           'accept': '*/*'}
URL[0]

'https://www.tripadvisor.com/Restaurant_Review-g187147-d1912643-Reviews-R_Yves-Paris_Ile_de_France.html'

In [66]:
rest_coord = []
coor = []

def get_html(url, params=None):
    r = requests.get(url, headers=HEADERS, params=params)
    return r


def get_coord(html):
    soup = BeautifulSoup(html, 'html.parser')
    coord_finds = soup.find(content = re.compile('https://maps.google.com/maps/api/'))
    print(coord_finds)
    coor.append(coord_finds)
    if coord_finds:
        coord_finds = coord_finds.get('content').split('&zoom')[0].split('?center=')[1].split('%2C')
    rest_coord.append(coord_finds)
    return coord_finds

def parse_coord(url):
    html = get_html(url)
    if html.status_code == 200:
        get_coord(html.text)
    else:
        print('Error')    
    
for i in range(1):
    parse_coord('https://google.com/maps/search/' + rest_pars[i]['coord_rest'])
    


<meta content="https://maps.google.com/maps/api/staticmap?center=48.8316299%2C2.3397946&amp;zoom=17&amp;size=256x256&amp;language=en&amp;sensor=false&amp;client=google-maps-frontend&amp;signature=d9Au1ZREtU_JLzDQqc6hCs528W8" itemprop="image"/>


NameError: name 'coord_finds' is not defined

In [67]:
rest_coord

[['48.8316299', '2.3397946']]

In [69]:
rest_pars

[{'meals': ['Lunch', 'Dinner', 'Brunch', 'Breakfast', 'Late Night'],
  'coord_rest': '17 boulevard Saint Jacques Paris Marriott Rive Gauche Hotel & Conference Center, 75014 Paris France'},
 {'meals': 'no_info',
  'coord_rest': '39 Aarstavaegen, Stockholm 120 52 Sweden'},
 {'meals': ['Lunch', 'Dinner', 'Late Night', 'Drinks'],
  'coord_rest': '30 North Audley Street Mayfair, London W1K 6ZF England'},
 {'meals': 'no_info', 'coord_rest': 'Berliner Str. 54, 13189 Berlin Germany'},
 {'meals': ['Dinner', 'Lunch'],
  'coord_rest': 'Zielstattstr. 6, 81379 Munich, Bavaria Germany'}]